#### Computer Vision - Fall 2020


## <span style="color:blue">Final Project - Rubik's cube mapper </span>

In this project we built a Rubik’s Cube mapper using classic computer vision techniques.


## Submission Notes:

1. In this project we used `python 3`, `numpy 1.18.5` and `cv2 4.4.0`
2. In order to use the code you should have a webcam and a Rubik's cube

## Instructions:

1. Run all the notebook and your webcam will open.
2. follow the written instructions. 
    1. find a good place to place the camera and the cube.
    2. Calibrate the colors by double clicking on the color on the screen
    3. Follow the printed instructions on how to rotate the cube
    4. If all went well you will see a printed version of the cube.
3. `active_cam` is the main function and has 1 parameter called `debug`.
    The default is `False`, but if `debug=True` you will see all of "behind the scene" screen and how the algorithm is actually working
4. If you want to recalibrate the colors you should restart the kernel and run all cells




In [1]:
import cv2
import numpy as np

In [2]:
import platform
print("Python version: ", platform.python_version())
print("Numpy version: ", np.__version__)
print("cv2 version: ", cv2.__version__)


Python version:  3.7.7
Numpy version:  1.18.5
cv2 version:  4.4.0


In [3]:
# Global param


clicked = False
frame = None

COLOR_NAMES = ['red', 'blue', 'green', 'yellow', 'orange', 'white']
COLOR_SIGNS = ['R', 'B', 'G', 'Y', 'O', 'W']
COLOR_IDX = 0

# COLOR_CALIBRATION dictionary that will hold the calibrated colors
# in the form of -> color name : 30x30x3 sample of the color (HSV) 
COLOR_CALIBRATION = {}

# COLOR_RANGE dictionary based on the COLOR_CALIBRATION
# that will hold for each color name his threshold of HSV values (lower and upper threshold)
# in the form of -> color name : (lower, upper)
# where lower and upper are hsv values -> (a, b ,c)
COLOR_RANGE = {}


COLOR_VERDICT = {}



# text style
font = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (50,50)
fontScale = 1
fontColor = (255,255,255)
lineType = 2




In [4]:
def update_res(frame, y, x):
    """
    Update the COLOR_CALIBRATION dictionary global param.
    where the key is the color name and the value
    is 30x30x3 sample of the color (in HSV format)
    
    Input:
    - frame: from the camera stream
    - y: with range of sub frame taken for color calibration  
    - x: high range of sub frame taken for color calibration 


    """
    pos = 15
    COLOR_CALIBRATION[COLOR_NAMES[COLOR_IDX]] = []
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # generates an hsv version of frame and 
                                                 # stores it in the hsv image variable
    COLOR_CALIBRATION[COLOR_NAMES[COLOR_IDX]] = hsv[y-pos:y+pos, x-pos:x+pos]



In [5]:
def mouse_click(event, x, y, flags, param):
    """
    the function is global action listener for mouse click that active  "update_res" on the calibration stage
    and also confirm validation of click on frame 
    
    
    Input:
    - event: action listener of mouse click on the frame
    - x: high range of sub frame taken for color calibration 
    - y: with range of sub frame taken for color calibration 
 

    """
    if event == cv2.EVENT_LBUTTONDBLCLK and frame is not None:
        global clicked
        clicked = True
        update_res(frame, y, x)


In [6]:
def stage_calibration(img):
    """
    the function activating the calibration stage every time a color is updated
    the user receive message on the screen what the color he need to calibrate by clicking to proper color 
    the function will continue until all 6 colors with be calibrated
    
    
    Input:
    - img: the frame from camera stream 
    -click: actionlistener of mouse click 
    
    output:
    -Updated :Global COLOR_IDX : count number of colors been updated 
    -frame +putText: frame with color need to calibrated
    
    """
    
    global COLOR_IDX
    global clicked
    if COLOR_IDX < 6:
        cv2.putText(img, COLOR_NAMES[COLOR_IDX].title(),(50,50),2,0.8,(255,255,255),2,cv2.LINE_AA)
    else:
        return
    
    if (clicked):
        clicked=False
        COLOR_IDX += 1
    return
        
    

In [7]:
def create_colors_range():
    """
    create the COLOR_RANGE dictionary global param based on
    the values in COLOR_CALIBRATION
    the key is the color name
    the value is (lower, upper) thresholds for HSV color range
    example -> 'W'  :  ((  0,   0, 215), (179,  55, 255))

    """
    global COLOR_RANGE
    
    # HSV values in the form of (a,b,c)
    # where 0 <=  a  <= 179
    #       0 <= b,c <= 255
    
    # factores to increase the range of the threshold
    factor_a = 10
    factor_bc = 25
    
    for color, values in COLOR_CALIBRATION.items():
        mins = values.min(axis=1).min(axis=0)
        maxs = values.max(axis=1).max(axis=0)

        a, b ,c = mins
        a = max(0, a-factor_a)
        b = max(0, b-factor_bc)
        c = max(0, c-factor_bc)
        min_th = (a, b, c)
        
        a, b ,c = maxs
        a = min(179, a+factor_a)
        b = min(255, b+factor_bc)
        c = min(255, c+factor_bc)
        max_th = (a, b, c)
        
        COLOR_RANGE[color[0].upper()] = (min_th, max_th)
    return

In [8]:
def crop_cube(image, thresh=175):
    """
    given an image the function will find the cube shape in its contour.
    
    Input:
    - image: the original frame
    - thresh: threshold 
    
    Returns:
    - original_contour: the original frame with the cube contour on top of it (image)
    - cropped: the cropped contour from the original image (image)
    - buffer: (x, y) the shift of the contour. will help draw the contours on the original frame

    """  
    orignel = image.copy()
    buffer = 0, 0
    kernel = np.ones((3,3),np.uint8)
        
    grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    grayImage = 255-grayImage

    grayImage = cv2.threshold(grayImage, thresh, 255, cv2.THRESH_BINARY)[1]

    
    opening = cv2.morphologyEx(grayImage, cv2.MORPH_OPEN, kernel, iterations=3)
    dilation = cv2.dilate(opening, kernel, iterations=3)
    closing = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel)
    
 
    contours, hierarchy = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntsSorted = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

    
    if cntsSorted != []:
        x, y, w, h = cv2.boundingRect(cntsSorted[0])

        original_contour = cv2.rectangle(orignel, (x,y), (x+w,y+h), (0,255,0), 2)    

        cropped = image[y: y+h, x: x+w]


        # buffer var helps as draw the contoures on the main frame
        buffer = x, y
    
    # unmark to see the filltered  (black and white frame)
#     cv2.imshow('filter_draw', filter_draw)

    blackwhite = cv2.cvtColor(closing, cv2.COLOR_GRAY2RGB)
    blackwhite = cv2.rectangle(blackwhite, (x,y), (x+w,y+h), (0,255,0), 2)


#     # unmark to add text of frame size to the cropped image
#     cv2.putText(cropped, 
#                 str(cropped.shape),
#                 bottomLeftCornerOfText, 
#                 font, 
#                 fontScale,
#                 fontColor,
#                 lineType)


    return  original_contour, cropped, buffer, blackwhite


In [9]:
def colors_mask(image):
    """
    the function will create a mask based on the colors in COLOR_RANGE
    
    Input:
    - image: the frame on which we want to create the mask
    
    Return:
    - result: the input image masked (the original pixels only where the mask is)

    """

    original = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = np.zeros((image.shape[0],image.shape[1]), dtype=np.uint8)
    
    for color, (lower, upper) in COLOR_RANGE.items():
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)
        
        # mark all the pixels that are in the range of the thresholds
        color_mask = cv2.inRange(image, lower, upper)
        mask = mask + color_mask
        
        # returns the original frame masked
        result = cv2.bitwise_or(original, original, mask=mask)
        
        #unmark to show the mask
#         cv2.imshow('color_mask_binary', mask)
        
    return result 

In [10]:
def find_contours(image, retrun_image=True):
    """
    the function will find the 9 largest contours in the image
    and returns list of their positions ordered LEFT TO RIGHT and TOP TO BOTTOM
    
    Input:
    - image: the frame (after colors_mask)
    - return_image: if True will return the image with the number contours on it
                    (help for debugging)
    
    Return:
    - final_list: list of the contours positions. each item in the list is (x, y, hight, width)
    - image: the input image with the 9 numbered contours on it


    """
    contours_positions = []
    lst = []
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntsSorted = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
    
    contours_positions = [cv2.boundingRect(contor) for contor in cntsSorted[:9]]
    
    
    # order the contours LEFT TO RIGHT and TOP TO BUTTOM
    #           1 2 3
    #           4 5 6
    #           7 8 9
    
    y_vals = sorted([l[1] for l in contours_positions])
    x_vals = sorted([l[0] for l in contours_positions])
    
    top_row = []
    mid_row = []
    low_row = []
    for l in contours_positions:
        if l[1] in y_vals[:3]:
            top_row.append(l)
        elif l[1] in y_vals[3:6]:
            mid_row.append(l)
        else:
            low_row.append(l)
    
    top_row = sorted(top_row, key=lambda x: x[0])
    mid_row = sorted(mid_row, key=lambda x: x[0])
    low_row = sorted(low_row, key=lambda x: x[0])
    
    final_list = top_row + mid_row + low_row
     
    if retrun_image:
        for i in range(len(final_list)):
            item = final_list[i]
            x, y, w, h = contours_positions[i]
            image = cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.putText(image, str(i+1), (item[0]+10, item[1]+10) ,font, fontScale,  fontColor, lineType)
        return final_list, image
    
    return final_list

In [11]:
def draw_contours(image, contours_positions):
    """
    the function will drow the 9 contours on the image
    
    Input:
    - image: the image on which we want to print the contours
    - contours_positions: list of the contours positions. each item -> (x, y, w, h)
    """
    for x, y, w, h in contours_positions:
        image = cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
    return image
    

In [12]:
def find_specific_color(contour):
    """
    the function will find the color of given contour (which is image)
    the colors are based on COLOR_RANGE
    Input:
    - contour: the image on which we want find her color
    
    Return:
    - decided_color_label: the sign of the decided color. if could not found and color from
                            COLOR_SIGNS will return 'X'
    """
    
    # Convert frame from RGB to HSV
    try:
        contour = cv2.cvtColor(contour, cv2.COLOR_BGR2HSV)
    except:
        return 'X'
    
    pixels_number = contour.shape[0]*contour.shape[1]
    decided_color_label = 'X'
    decided_color_value = 0
    
    # the ratio will idicate which color has the most pixels in the contour
    # we will find the biggest color ratio and return it
    ratio = 0
    
    for color, (lower, upper) in COLOR_RANGE.items():
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)
        try:
            mask = cv2.inRange(contour, lower, upper)
            ratio = (mask == 255).sum() / pixels_number
        except:
            mask = np.zeros(contour.shape)

        if ratio > decided_color_value:
            decided_color_value = ratio
            decided_color_label = color[0].upper()
    
    return decided_color_label

In [13]:
def find_colors(frame, contours_positions):
    
    """
    the function will find the colors on the contours in the frame
    it will sample each contour from the frame and find his color using find_specific_color function
    
    Input:
    - frame: the frame on which we want to find the colors
    - contours_positions: list of the contours_positions 
    Return:
    - frame: the input frame with the color classification printed o it
    - classified: string (length 9) represent the colors that were found in the frame

    """
    colors_names = []
    s = 5
    for x, y, w, h in contours_positions:
        contour = frame[y+s: y+h-s, x+s: x+w-s]
        name = find_specific_color(contour)
        colors_names.append(name)

        
    classified = "".join(colors_names)
    
    
    for i in range(len(colors_names)):
        x, y, w, h = contours_positions[i]
#         image = cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
        frame = cv2.putText(frame, colors_names[i], (x, y) ,font, fontScale,  fontColor, lineType)
    return frame, classified

In [14]:
def find_most_freq(freq_list):
    """
    the function will find the most frequent string (that represent the colors verdicts)
    in given list
    
    Input:
    - freq_list: list with string representing the colors verdicts
    Return:
        prints the user instruction and eventually the virtual cube

    """
    unique, pos = np.unique(np.array(freq_list), return_inverse=True)
    counts = np.bincount(pos)
    maxpos = counts.argmax()
    verdict = unique[maxpos]
    
    # checks that the output is valid
    # if yes returns the face and the verdict
    # otherwise return 'X' and verdict
    # 'X' represnt failure
    
    if len(verdict) == 9 and 'X' not in verdict:
        face = verdict[4]
        return face, verdict
    else:
        return 'X', verdict

In [15]:
def create_cube_string(order):
    """
    the function will print representation of the cube.
    The instructions for the user are:
    rotate left 4 times, then up 1 time and then down 2 times. 
    on the right cube you can see the order of the instruction.
    
        WWW                   555
        WWW                   555
        WWW                   555
    OOO GGG RRR BBB       111 444 333 222 
    OOO GGG RRR BBB       111 444 333 222
    OOO GGG RRR BBB       111 444 333 222
        YYY                   666
        YYY                   666
        YYY                   666
    
    Input:
    - order:list in length 6 that represent the order of the colors
    """
    global COLOR_VERDICT
    
    strings = [COLOR_VERDICT[c] for c in order]
    
    for i in range(3):
        print('    ' + strings[4][i*3:(i+1)*3])

    temp = [strings[:-2][0], strings[:-2][3], strings[:-2][2], strings[:-2][1]]
    for i in range(3):
        string = ''
        for l in temp:
            string += (l[i*3:(i+1)*3])+ ' '
        print(string)
            
    
    for i in range(3):
        print('    ' + strings[5][i*3:(i+1)*3])
    

In [16]:
def active_cam(debug=False):
    """
    This is the main function. it will open the video camera, capture the video and call all other functions
    
    Input:
    - debug: if True will show all "behind the scenes" processing

    """
    cap = cv2.VideoCapture(0)
    cv2.namedWindow('frame')
    cv2.setMouseCallback('frame', mouse_click)

    try:
        
        verdicts = []
        temp_values = ''
        global order
        order = ['X','X','X','X','X','X']
        order_idx = 0
        
        while(True):
            global frame
            global COLOR_VERDICT
            # Capture frame-by-frame
            ret, frame = cap.read()
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            stage_calibration(frame)
        
            cv2.imshow('frame', frame)
            
            if len(COLOR_CALIBRATION) < 6:
                continue
                          
            create_colors_range()       
                    
            # crop the image
            frame_contour, frame_cropped, buffer, frame_blackwhite_mask = crop_cube(frame, thresh=170)
            
            if debug:
                cv2.imshow('frame_contour', frame_contour)
                cv2.imshow('frame_cropped', frame_cropped)
                cv2.imshow('frame_blackwhite_mask', frame_blackwhite_mask)

            # create the color mask
            frame_cropped_colors_mask = colors_mask(frame_cropped)
            
            if debug: 
                cv2.imshow('frame_cropped_colors_mask', frame_cropped_colors_mask)

            # find contours using the color mask
            contours_positions_cropped, frame_cropped_contours = find_contours(frame_cropped_colors_mask)
            
            if debug:
                cv2.imshow('frame_cropped_contours', frame_cropped_contours)
            
            # create positions list considering the main frame
            contours_positions = [(buffer[0]+x, buffer[1]+y, h, w) for x, y, h, w in contours_positions_cropped]
            
            # show the contours on the original frame
            frame = draw_contours(frame, contours_positions)
            
            frame_colors_verdicts, colors_verdict = find_colors(frame, contours_positions)
            cv2.imshow('frame', frame_colors_verdicts)
            
            verdicts.append(colors_verdict)
            # every 35 stream of frames (betch) choose the classifaction
            # there is some tests to make sure the input is valid
            # if tow betchs are the same we will appned the clasiffcation verdict
            if len(verdicts) == 35:
                face, values = find_most_freq(verdicts)
                if face != 'X': # make sure we found a real color as face
                    if temp_values == values: 
                        if face not in COLOR_VERDICT:
                            COLOR_VERDICT[face] = values
                            order[order_idx] = face
                            order_idx += 1
                            print(f'found faces: {order}')
                        
                        # in this part there is prints guding the user
                        remain_colors = set(COLOR_SIGNS)-set(COLOR_VERDICT.keys())
                        if len(remain_colors) > 2: 
                            print('rotate left')
                        elif len(remain_colors) == 2:
                            print('rotate up')
                        elif len(remain_colors) == 1:
                            print('rotate down twice')
                        else:
                            create_cube_string(order)
                            break
                    else:
                        print('wait')
                        
                temp_values = values
                verdicts = []
                
                
    finally:
        # When everything done, release the capture
        cap.release()
        cv2.destroyAllWindows()

In [17]:
active_cam(debug=False)

wait
found faces: ['Y', 'X', 'X', 'X', 'X', 'X']
rotate left
rotate left
wait
found faces: ['Y', 'O', 'X', 'X', 'X', 'X']
rotate left
rotate left
rotate left
wait
found faces: ['Y', 'O', 'W', 'X', 'X', 'X']
rotate left
rotate left
rotate left
wait
found faces: ['Y', 'O', 'W', 'R', 'X', 'X']
rotate up
rotate up
rotate up
rotate up
rotate up
wait
wait
wait
wait
wait
found faces: ['Y', 'O', 'W', 'R', 'G', 'X']
rotate down twice
rotate down twice
rotate down twice
wait
wait
wait
found faces: ['Y', 'O', 'W', 'R', 'G', 'B']
    YOY
    YGW
    YYB
GBG OGW RBO BWR 
OYW GRO GWR GOB 
RRO BYG ORR BRW 
    WOW
    WBY
    GBY


THE END